### The temporal network class

In this notebook we explore how to load the coherence connectivity data with the temporal network class

In [2]:
# Path to the toolbox
import sys; sys.path.insert(1, '/home/vinicius/storage1/projects/GrayData-Analysis')
import xarray
# GDa
from   GDa.temporal_network import temporal_network

### Instantiate temporal network

In [5]:
# Parameters to read the data
idx      = 3 
nses     = 1
nmonkey  = 0
align_to = 'cue'

dirs = { 'rawdata':'/home/vinicius/storage1/projects/GrayData-Analysis/GrayLab/',
         'results':'Results/',
         'monkey' :['lucy', 'ethyl'],
         'session':'session01',
         'date'   :[['141014', '141015', '141205', '150128', '150211', '150304'], []] }

In [ ]:
# Instantiating a temporal network object without thresholding the data
net =  temporal_network(coh_file="super_tensor_k_500_surr_False_morlet.nc ", monkey=dirs['monkey'][nmonkey], 
                        session=1, date='150128', trial_type=[1],
                        behavioral_response=[1], wt=(20,20), drop_trials_after=True,
                        verbose=True, q=None)

In [6]:
from xfrites.conn.conn_coh import conn_coherence_wav

In [7]:
# Defining parameters
f_start, f_end, n_freqs, sfreq = .1, 80, 50, ses.data.attrs['fsample']
freqs = np.linspace(f_start, f_end, n_freqs, endpoint=True)
delta = 15 # Downsampling factor
mode  = 'morlet' # ("morlet", "mt_1", "mt_2")
if mode in ["morlet", "mt_1"]:
    foi   = np.array([
            [0.1, 6.],
            [6., 14.],
            [14., 26.],
            [26., 42.],
            [42., 80.]
                ])
    n_cycles     = freqs/2
    mt_bandwidth = None
    decim_at='tfd'
elif mode == "mt_2":
    foi   = np.array([
            [0.1, 6.],
            [6., 14.],
            [14., 26.],
            [26., 42.],
            [42., 80.]
                ])
    freqs = foi.mean(axis=1)
    W     = np.ceil( foi[:,1]-foi[:,0] )   # Bandwidth
    foi   = None     
    n_cycles     = np.array([3, 5, 9, 12, 16])
    mt_bandwidth = np.array([2, 4, 4.28, 5.647, 9.65])
    decim_at     = 'coh'

In [8]:
ses.data.roi.astype('<U13')

<xarray.DataArray 'roi' (roi: 49)>
array(['a46V', 'a24c', 'a46D', 'F7', 'a9/46D', 'a8M', 'a8r', 'a8M', 'F2',
       'F2', 'a44', 'F2', 'a8L', 'F2', 'F2', 'F1', 'a3', 'F1', 'F1', 'F1',
       'a3', 'a7B', 'a7B', 'a7B', 'Thal', 'a3', 'a7B', 'a7A', 'LIP',
       'LIP', 'a5', 'a5', 'a5', 'a7A', 'a5', 'MT', 'DP', 'DP', 'V6A',
       'DP', 'V2', 'V6A', 'V1', 'V1', 'V1', 'V2', 'V1', 'V1', 'V1'],
      dtype='<U13')
Coordinates:
  * roi      (roi) <U13 'a46V' 'a24c' 'a46D' 'F7' ... 'V2' 'V1' 'V1' 'V1'

In [9]:
kw = dict(
    freqs=freqs, times=ses.data.time, roi=ses.data.roi, foi=foi, n_jobs=-1,
    sfreq=sfreq, mode=mode, decim_at=decim_at, n_cycles=n_cycles, decim=delta,
    sm_times=300, sm_freqs=1, block_size=2
)

# compute the coherence
coh = conn_coherence_wav(ses.data, **kw)

Compute pairwise coherence (n_pairs=1176, n_freqs=50, decim=15)
[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  20 out of  49 | elapsed:   10.7s remaining:   15.5s
[Parallel(n_jobs=40)]: Done  30 out of  49 | elapsed:   12.9s remaining:    8.2s
[Parallel(n_jobs=40)]: Done  40 out of  49 | elapsed:   15.1s remaining:    3.4s
[Parallel(n_jobs=40)]: Done  49 out of  49 | elapsed:   17.1s finished


  0%|          |  : 0/1176 [00:00<?,       ?it/s]

[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  20 out of  49 | elapsed:   15.5s remaining:   22.5s
[Parallel(n_jobs=40)]: Done  30 out of  49 | elapsed:   17.7s remaining:   11.2s
[Parallel(n_jobs=40)]: Done  40 out of  49 | elapsed:   19.9s remaining:    4.5s
[Parallel(n_jobs=40)]: Done  49 out of  49 | elapsed:   21.8s finished


  0%|          |  : 0/1176 [00:00<?,       ?it/s]

In [12]:
A = coh.stack(observations=("trials","times"))

In [11]:
from frites.conn import conn_reshape_undirected

In [18]:
A =conn_reshape_undirected(coh.isel(trials=0, freqs=0), fill_diagonal=1, verbose=True)

WARNING | Duplicated entries found and removed : [('a46V', 'a8M') ('F2', 'a46V') ('F2', 'a46V') ... ('V1', 'V1')
 ('V1', 'V1') ('V1', 'V1')]


In [44]:
for i in range(A.sizes["times"]):
    plt.figure(figsize=(12,10))
    plt.imshow(A.isel(times=i), origin="lower", vmin=0, vmax=1)
    plt.yticks((range(A.sizes["sources"])),A.sources.values, fontsize=12)
    plt.xticks((range(A.sizes["sources"])),A.sources.values, rotation=270, fontsize=12)
    plt.colorbar()
    plt.savefig(f'img/a_{i}.png')
    plt.close()

In [22]:
A

<xarray.DataArray 'coh' (sources: 22, targets: 22, times: 244)>
array([[[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.8686624 , 0.85889851, 0.85324091, ..., 0.80014675,
         0.81358377, 0.82753161],
        [0.93098043, 0.92797511, 0.92592005, ..., 0.64760866,
         0.68020408, 0.70438485],
        ...,
        [0.93561742, 0.92563627, 0.91420074, ..., 0.81666482,
         0.83438343, 0.85285902],
        [0.7997576 , 0.79228266, 0.79349012, ..., 0.92170269,
         0.94870854, 0.96763015],
        [0.91847344, 0.91083309, 0.90449519, ..., 0.90946458,
         0.9387025 , 0.95782646]],

       [[0.8686624 , 0.85889851, 0.85324091, ..., 0.80014675,
         0.81358377, 0.82753161],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.96911858, 0.96336635, 0.95816707, ..., 0.83102548,
         0.88373099, 0.91271683],
...
        [0.87986188, 0.86518186, 0.85703219, ..., 0.72459055,
         0.756868  , 0.79174063],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.94449317, 0.93729336, 0.93388268, ..., 0.76078625,
         0.84196132, 0.89928808]],

       [[0.91847344, 0.91083309, 0.90449519, ..., 0.90946458,
         0.9387025 , 0.95782646],
        [0.98762585, 0.98533869, 0.98300366, ..., 0.68876992,
         0.70943291, 0.73118235],
        [0.98777979, 0.98526173, 0.98309865, ..., 0.46706148,
         0.52957049, 0.57419181],
        ...,
        [0.97758702, 0.97513274, 0.97324652, ..., 0.71224051,
         0.73560236, 0.76109128],
        [0.94449317, 0.93729336, 0.93388268, ..., 0.76078625,
         0.84196132, 0.89928808],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ]]])
Coordinates:
  * sources  (sources) <U6 'a24c' 'a46D' 'F7' 'a46V' ... 'a7A' 'a5' 'a9/46D'
  * targets  (targets) <U6 'a24c' 'a46D' 'F7' 'a46V' ... 'a7A' 'a5' 'a9/46D'
    trials   int64 0
    freqs    float64 3.361
  * times    (times) float64 -0.65 -0.635 -0.62 -0.605 ... 2.95 2.965 2.98 2.995
Attributes:
    sfreq:         1000.0
    sm_times:      20
    sm_freqs:      1
    sm_kernel:     hanning
    mode:          morlet
    n_cycles:      [ 0.05        0.86530612  1.68061224  2.49591837  3.311224...
    mt_bandwidth:  None
    decim:         15

In [48]:
coh.to_netcdf("data/first_array.nc")

In [49]:
xr.load_dataarray("data/first_array.nc")

<xarray.DataArray 'coh' (trials: 750, roi: 1176, freqs: 5, times: 244)>
array([[[[0.58553699, 0.57608337, 0.56195629, ..., 0.87903509,
          0.8969719 , 0.91427853],
         [0.71571351, 0.71288565, 0.71903749, ..., 0.75857962,
          0.79109621, 0.8126539 ],
         [0.83379748, 0.81145263, 0.78906552, ..., 0.70979858,
          0.72421791, 0.73816791],
         [0.63017598, 0.61882486, 0.61733093, ..., 0.80188807,
          0.80407003, 0.81028333],
         [0.85247666, 0.81517685, 0.77298798, ..., 0.8101433 ,
          0.84349445, 0.87171904]],

        [[0.68708444, 0.68111085, 0.67150739, ..., 0.94833125,
          0.94298934, 0.93784511],
         [0.78609878, 0.7614654 , 0.74527956, ..., 0.69860661,
          0.73216801, 0.75600205],
         [0.77195556, 0.78492539, 0.79578828, ..., 0.61783205,
          0.66672884, 0.71263371],
         [0.73150587, 0.69427043, 0.66167614, ..., 0.83761288,
          0.84958619, 0.86213974],
         [0.90326385, 0.87879619, 0.85182184, ..., 0.80671753,
...
          0.97610685, 0.97040773],
         [0.97432761, 0.96535403, 0.95457178, ..., 0.84247592,
          0.83930509, 0.84726967],
         [0.91667973, 0.88521853, 0.84591491, ..., 0.80191183,
          0.78055348, 0.76760434],
         [0.92296476, 0.90599455, 0.88692144, ..., 0.73980144,
          0.77154248, 0.80249719],
         [0.86771029, 0.84031823, 0.80974223, ..., 0.67389209,
          0.71140262, 0.75005255]],

        [[0.90300754, 0.85683168, 0.80295985, ..., 0.90258334,
          0.88578804, 0.86663768],
         [0.97370564, 0.96324951, 0.95005657, ..., 0.49974761,
          0.55455331, 0.62783912],
         [0.90459337, 0.86766334, 0.82211268, ..., 0.63758581,
          0.61249609, 0.59990919],
         [0.89533848, 0.87025125, 0.84019127, ..., 0.6829391 ,
          0.71121549, 0.73991646],
         [0.86131388, 0.8332347 , 0.80316789, ..., 0.58283564,
          0.63890697, 0.70259375]]]])
Coordinates:
  * roi      (roi) object 'a24c-a46V' 'a46D-a46V' 'F7-a46V' ... 'V1-V1' 'V1-V1'
  * trials   (trials) int32 0 1 2 3 9 12 13 ... 1000 1001 1002 1003 1004 1005
  * times    (times) float64 -0.65 -0.635 -0.62 -0.605 ... 2.95 2.965 2.98 2.995
  * freqs    (freqs) float64 3.361 10.7 20.48 34.34 61.25
Attributes:
    sfreq:      1000.0
    sm_times:   20
    sm_freqs:   1
    sm_kernel:  hanning
    mode:       morlet
    n_cycles:   [ 0.05        0.86530612  1.68061224  2.49591837  3.31122449 ...
    decim:      15

In [60]:
coh.attrs = {}

In [59]:
for key in keys: 
    del coh.attrs[key] 

RuntimeError: dictionary changed size during iteration